In [3]:
import os

formats_to_convert = [".m4a"]

if not os.path.exists("VoxCeleb_gender"):
    !pip install gdown pydub librosa
    !gdown https://drive.google.com/uc?id=1HRbWocxwClGy9Fj1MQeugpR4vOaL9ebO
    !unzip -q VoxCeleb_gender.zip
    !rm VoxCeleb_gender.zip
    !rm -r __MACOSX

    from pydub import AudioSegment

    for (dirpath, dirnames, filenames) in os.walk("VoxCeleb_gender/females/"):
        for filename in filenames:
            if filename.endswith(tuple(formats_to_convert)):

                filepath = dirpath + "/" + filename
                (path, file_extension) = os.path.splitext(filepath)
                file_extension_final = file_extension.replace(".", "")
                try:
                    track = AudioSegment.from_file(filepath, file_extension_final)
                    wav_filename = filename.replace(file_extension_final, "wav")
                    wav_path = dirpath + "/" + wav_filename
                    print("CONVERTING: " + str(filepath))
                    file_handle = track.export(wav_path, format="wav")
                    os.remove(filepath)
                except:
                    print("ERROR CONVERTING " + str(filepath))
    for (dirpath, dirnames, filenames) in os.walk("VoxCeleb_gender/males/"):
        for filename in filenames:
            if filename.endswith(tuple(formats_to_convert)):

                filepath = dirpath + "/" + filename
                (path, file_extension) = os.path.splitext(filepath)
                file_extension_final = file_extension.replace(".", "")
                try:
                    track = AudioSegment.from_file(filepath, file_extension_final)
                    wav_filename = filename.replace(file_extension_final, "wav")
                    wav_path = dirpath + "/" + wav_filename
                    print("CONVERTING: " + str(filepath))
                    file_handle = track.export(wav_path, format="wav")
                    os.remove(filepath)
                except:
                    print("ERROR CONVERTING " + str(filepath))
else:
    print("Dataset already downloaded.")

Dataset already downloaded.


In [5]:
 !pip install gdown pydub librosa

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.1/385.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 10.2 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 3.

In [33]:
import librosa
import scipy.signal as ss

def extract_mfcc(file_path, n_mfcc=12, win_length=320, hop_length=160):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    if sr != 16000:
        wave = librosa.resample(wave, sr, 16000)
    mfcc = librosa.feature.mfcc(y=wave, sr=16000, n_mfcc=n_mfcc, window=ss.windows.hamming, win_length=win_length, hop_length=hop_length)
    return mfcc

# Build data in memory

In [29]:
import pandas as pd

aux = []

for (dirpath, dirnames, filenames) in os.walk("VoxCeleb_gender"):
    for filename in filenames:
        if filename.endswith(".wav"):
            filepath = dirpath + "/" + filename
            aux.append({"file": filepath, "gender": dirpath.split("/")[-1]})

df = pd.DataFrame(aux)
            

In [16]:
display(df)

,file,gender
0,VoxCeleb_gender/males/255.wav,males
1,VoxCeleb_gender/males/1908.wav,males
2,VoxCeleb_gender/males/2965.wav,males
3,VoxCeleb_gender/males/2733.wav,males
4,VoxCeleb_gender/males/3091.wav,males
...,...,...
5988,VoxCeleb_gender/females/489.wav,females
5989,VoxCeleb_gender/females/941.wav,females
5990,VoxCeleb_gender/females/63.wav,females
5991,VoxCeleb_gender/females/1820.wav,females


In [34]:
extract_mfcc(df.loc[i, "file"]).shape

(12, 423)

In [35]:

for i in range(len(df)):
    df.loc[i, "mfcc"] = extract_mfcc(df.loc[i, "file"])

ValueError: could not broadcast input array from shape (12,423) into shape (423,)